In [20]:
import pandas as pd
import os

In [21]:
data_path = '/Users/ryan/Documents/GitHub/MGT4187-Project/data'
os.chdir(data_path)

data = pd.read_csv('matched_disney_dataset.csv', index_col=0)
data.columns

Index(['imdb_id', 'title', 'plot', 'type', 'rated', 'year', 'released_at',
       'added_at', 'runtime', 'genre', 'director', 'writer', 'actors',
       'language', 'country', 'awards', 'metascore', 'imdb_rating',
       'imdb_votes', 'production_budget', 'domestic_gross', 'worldwide_gross',
       'distributor', 'mpaa_rating', 'matched_profit_indicator', 'Network',
       'Critic Score', 'Audience Score', 'tv_link', 'nob_audience', 'Rating',
       'Review_audience', 'nob_critics', 'Sentiment', 'Review_critics',
       'matched_audience_indicator', 'matched_critics_indicator',
       'matched_all_review_indicator'],
      dtype='object')

In [22]:
print('total review data:', data['matched_all_review_indicator'].sum())

total review data: 19.0


In [23]:
data = data.drop_duplicates(subset='imdb_id')
data = data.dropna(subset='imdb_rating')

In [24]:
vlist = ['imdb_id', 'title', 'plot', 'type', 'rated', 'year', 'released_at',
       'added_at', 'runtime', 'genre', 'director', 'writer', 'actors',
       'language', 'country', 'awards', 'metascore', 'imdb_rating',
       'imdb_votes']
data = data[vlist]

## data supplement

In [25]:
import re
from cleanco import basename

### supplement from rank 1-500 movies

In [26]:
supplement_data_path = '/Users/ryan/Documents/GitHub/MGT4187-Project/data/supplement_profit_and_production'
os.chdir(supplement_data_path)

supplement1 = pd.read_csv('supplement_production_profit_1_500.csv')

In [27]:
## clean name function
def std_name(v):
    try:
        v = basename(basename(
            v.lower()))  # change string to lower case, then get the base name applying basename to get rid of common name suffixes
        v = re.sub(' +', ' ', re.sub('^\W*the', ' ', v)).strip()  # get rid of "the" at the begining of names
        v = re.sub(r'[^a-zA-Z\s]', '', v)  # delete the non-character symbols
        return v
    except:
        return v

In [28]:
data['title_std'] = data['title'].apply(std_name)
supplement1['title_std'] = supplement1['Movie'].apply(std_name)


combined_new = pd.merge(data, supplement1[['title_std', 'Movie', 'Production Budget',
       'Domestic Gross', 'Worldwide Gross']], on='title_std', how='left', indicator='profit_indicator')

In [29]:
combined_new['profit_indicator'].value_counts()

profit_indicator
left_only     792
both           96
right_only      0
Name: count, dtype: int64

### Supplement from Kaggle dataset
Source: https://www.kaggle.com/datasets/sujaykapadnis/horror-movies-profits-dataset

In [30]:
os.chdir(data_path)

supplement2 = pd.read_csv('movie_profit.csv')

In [34]:
data2 = combined_new.loc[combined_new['profit_indicator']=='left_only']
data2 = data2[vlist]
data2['title_std'] = data2['title'].apply(std_name)
supplement2['title_std'] = supplement2['movie'].apply(std_name)


combined_new2 = pd.merge(data2, supplement2[['title_std', 'movie', 'production_budget',
       'domestic_gross', 'worldwide_gross']], on='title_std', how='left', indicator='profit_indicator')

In [35]:
combined_new2['profit_indicator'].value_counts()

profit_indicator
left_only     699
both           94
right_only      0
Name: count, dtype: int64